# 文件夹目录

这部分介绍如何使用 `DirectoryLoader` 加载文件夹中的所有文档。默认情况下，底层使用 [UnstructuredLoader](./unstructured_file.ipynb)。

In [1]:
from langchain.document_loaders import DirectoryLoader

我们可以使用 `glob` 参数控制要加载哪些文件。注意这里不会加载 `.rst` 文件或者 `.ipynb` 文件。

In [2]:
loader = DirectoryLoader('../', glob="**/*.md")

In [3]:
docs = loader.load()

In [4]:
len(docs)

1

## 显示进度条

默认情况下不会显示进度条。要显示进度条，请安装 `tqdm` 库（例如 `pip install tqdm`），并将 `show_progress` 参数设置为 `True`。

In [10]:
%pip install tqdm
loader = DirectoryLoader('../', glob="**/*.md", show_progress=True)
docs = loader.load()

0it [00:00, ?it/s]


## 使用多线程

默认情况下，加载只会在一个线程中进行。要使用多个线程，请将 `use_multithreading` 标志设置为 `True`。

In [ ]:
loader = DirectoryLoader('../', glob="**/*.md", use_multithreading=True)
docs = loader.load()

## 更改加载器类
默认情况下使用 `UnstructuredLoader` 类。然而，您可以很容易地更改加载器的类型。

In [15]:
from langchain.document_loaders import TextLoader

In [6]:
loader = DirectoryLoader('../', glob="**/*.md", loader_cls=TextLoader)

In [7]:
docs = loader.load()

In [8]:
len(docs)

1

如果需要加载Python源代码文件，请使用 `PythonLoader`。

In [14]:
from langchain.document_loaders import PythonLoader

In [13]:
loader = DirectoryLoader('../../../../../', glob="**/*.py", loader_cls=PythonLoader)

In [14]:
docs = loader.load()

In [15]:
len(docs)

691

## 使用 TextLoader 自动检测文件编码

在这个例子中，我们将展示如何使用 `TextLoader` 类从目录中加载任意数量的任意类型的文件时，一些有用的策略。

首先为了说明这个问题，让我们试着加载多个使用任意编码的文本文件。

In [16]:
path = '../../../../../tests/integration_tests/examples'
loader = DirectoryLoader(path, glob="**/*.txt", loader_cls=TextLoader)

### A. 默认行为

In [19]:
loader.load()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /data/source/langchain/langchain/document_loaders/text.py:29 in load                             │
│                                                                                                  │
│   26 │   │   text = ""                                                                           │
│   27 │   │   with open(self.file_path, encoding=self.encoding) as f:                             │
│   28 │   │   │   try:                                                                            │
│ ❱ 29 │   │   │   │   text = f.read()                                                             │
│   30 │   │   │   except UnicodeDecodeError as e:                                                 │
│   31 │   │   │   │   if self.autodetect_encoding:                                                │
│   32 │   │   │   │   │   detected_encodings = self.detect_file_encodings()                       │
│                                                                                                  │
│ /home/spike/.pyenv/versions/3.9.11/lib/python3.9/codecs.py:322 in decode                         │
│                                                                                                  │
│    319 │   def decode(self, input, final=False):                                                 │
│    320 │   │   # decode input (taking the buffer into account)                                   │
│    321 │   │   data = self.buffer + input                                                        │
│ ❱  322 │   │   (result, consumed) = self._buffer_decode(data, self.errors, final)                │
│    323 │   │   # keep undecoded input until the next call                                        │
│    324 │   │   self.buffer = data[consumed:]                                                     │
│    325 │   │   return result                                                                     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xca in position 0: invalid continuation byte

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 loader.load()                                                                                │
│   2                                                                                              │
│                                                                                                  │
│ /data/source/langchain/langchain/document_loaders/directory.py:84 in load                        │
│                                                                                                  │
│   81 │   │   │   │   │   │   if self.silent_errors:                                              │
│   82 │   │   │   │   │   │   │   logger.warning(e)                                               │
│   83 │   │   │   │   │   │   else:                                                               │
│ ❱ 84 │   │   │   │   │   │   │   raise e                                                         │
│   85 │   │   │   │   │   finally:                                                                │
│   86 │   │   │   │   │   │   if pbar:                                                            │
│   87 │   │   │   │   │   │   │   pbar.update(1)                                                  │
│                                                                                                  │
│ /data/source/langchain/langchain/document_loaders/directory.py:78 in load                   

文件 `example-non-utf8.txt` 使用了不同的编码，`load()` 函数会因为解码失败而抛出一个有用的错误信息。

使用 `TextLoader` 的默认行为，如果任何一个文档加载失败，整个加载过程都会失败，并且不会加载任何文档。

### B. 静默失败

我们可以给 `DirectoryLoader` 传递 `silent_errors` 参数以跳过加载失败的文件并继续加载过程。

In [30]:
loader = DirectoryLoader(path, glob="**/*.txt", loader_cls=TextLoader, silent_errors=True)
docs = loader.load()

Error loading ../../../../../tests/integration_tests/examples/example-non-utf8.txt


In [35]:
doc_sources = [doc.metadata['source']  for doc in docs]
doc_sources

['../../../../../tests/integration_tests/examples/whatsapp_chat.txt',
 '../../../../../tests/integration_tests/examples/example-utf8.txt']

### C. 自动检测编码

我们也可以要求 `TextLoader` 在失败之前自动检测文件编码，只需将 `autodetect_encoding` 传递给加载器类。

In [37]:
text_loader_kwargs={'autodetect_encoding': True}
loader = DirectoryLoader(path, glob="**/*.txt", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
docs = loader.load()


In [38]:
doc_sources = [doc.metadata['source']  for doc in docs]
doc_sources

['../../../../../tests/integration_tests/examples/example-non-utf8.txt',
 '../../../../../tests/integration_tests/examples/whatsapp_chat.txt',
 '../../../../../tests/integration_tests/examples/example-utf8.txt']